In [ ]:
# !pip install bert-extractive-summarizer
# !pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q tensorflow-gpu==2.2

import os
os.environ["CUDA_VISIBLE_DEVICES"]= "1"
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np

from pathlib import Path

--------------------------------------------------

DATA LOADER

In [ ]:
import pandas as pd
check_path = "/content/train.csv"
my_file = Path(check_path)
flag = 0
if flag and not my_file.is_file():
    check_path = "/content/AMICorpusXML/data/ami-summary/abstractive/ES2004a.abssumm.txt"

    my_file = Path(check_path)
    if not my_file.is_file():
        if not Path("/content/AMICorpusXML/data").is_dir():
            ! git clone https://github.com/gcunhase/AMICorpusXML
        ! python /content/AMICorpusXML/main_obtain_meeting2summary_data.py --summary_type abstractive
    else:
        print("Data already prepared... Importing\n")

    val = "ES2003, ES2011, IS1008, TS3004, TS3006".split(',')
    train = """ES2002, ES2005, ES2006, ES2007, ES2008, ES2009, ES2010, ES2012, ES2013, ES2015, 
                ES2016, IS1000, IS1001, IS1003, IS1004, IS1006, IS1007, TS3005, TS3008, TS3009, TS3010, 
                TS3011, TS3012""".split(",")
    test = "ES2004, ES2014, IS1009, TS3003, TS3007".split(",")

    story_directory = r'/content/AMICorpusXML/data/ami-transcripts-stories/abstractive'
    sum_directory = r'/content/AMICorpusXML/data/ami-summary/abstractive'

    story_val = []
    for filename in os.listdir(story_directory):
        if filename.endswith(".story"):
            for each in val:
                if each.strip() in str(filename):
                    story_val.append(filename)

    def data_read(f1):
        with open(f1, 'r') as file:
            data1 = file.read().replace('\n', '')
        return data1

    val_data = []
    val_summ = []
    for each in story_val:
        a = each.split('.')[0]
        story_file = story_directory+'/'+each
        summ_file = sum_directory+'/'+a+".abssumm.txt"
        data = data_read(story_file)
        val_data.append(data)
        data = data_read(summ_file)
        val_summ.append(data)
else:
    train = pd.read_csv("./content/train.csv", sep='\t')
    train = train.drop(['Unnamed: 0'],axis=1)
    test = pd.read_csv("./content/test.csv", sep='\t')
    test = test.drop(['Unnamed: 0'],axis=1)
    val = pd.read_csv("./content/val.csv", sep='\t')
    val = val.drop(['Unnamed: 0'],axis=1)

In [ ]:
train.head()

----------------------

## EXTRACTIVE SUMMARY GENERATOR

BERTSUM

In [ ]:
from summarizer import Summarizer
from tqdm import tqdm
model = Summarizer()

In [ ]:
# test = test[]

In [ ]:
ext_summary = []
# model.eval()
for index, row in tqdm(train.iterrows(), ):
    input = row['inputs']
    target = row['targets']
#     print(input)
    result = model(input, max_length=512)
#     summ = "".join(result)
    ext_summary.append((result,target))

# test['ext_summary'] = ext_summary

In [ ]:
len(input.split()), len(result.split())

In [ ]:
result

In [10]:
m = 3
a = [train_[i:i+m] for i in range(0,len(train_),m)]

In [14]:
len(a[0])

3

In [15]:
train_[0][0]

"Um, basically what I'm gonna have to talk to you about today is um component design and it's been brought to my attention that we may be somewhat limited as to what we can do because of what our manufacturer offers, so Basically what I'm gonna be doing is talking to you about that. Um, components of a remote control, okay. Um, and also there's a kinetic energy possibility. And if we do go with the rubber doubled curved case um we'll have to use rubber push buttons because the other buttons aren't compatible with that. And if you were to record if you were to make a video recording you could actually see the light. Um, of course Yeah, just as a fun gimmick. Um, and then on to the circuit board that we're gonna use, also known as the chip. Um, if we do go the lithium battery route then we'll have to go outside our current manufacturer. It is the new it would be in a class of its own. We could initially go with what we have and if we can find them cheaper later on Yeah. Doesn't have to b

In [16]:
for i in a:
    title = [j[1] for j in i]
    body = [j[0] for j in i]

In [18]:
title[0]

'The project manager opened the meeting and read the minutes of the previous meeting. The industrial designer and user interface designer presented the prototype they created, which was designed to look like a banana. The marketing expert conducted an evaluation of the prototype. The team found that, although the overall design of the prototype was attractive, its yellow color was ugly. The team rated the prototype highly on its ease of use and felt that its yellow color and shape detracted slightly from its ability to be misplaced and that a feature which causes the remote to make noise based on its proximity to a television needed to be added. The team thought the prototype was fashionable and not technologically innovative or spongy. The project manager led the team in calculating the production costs of the remote and ensuring that they aligned with the project budget. The costs were over budget, so the team opted to exclude the LCD from their design to meet their budget. The team 

In [11]:
len(train_)

92

In [13]:
len(a), len(a)*m

(31, 93)

In [3]:
import json
with open ('../Ext_Abs/train.txt', "r") as f:
    train_ = json.loads(f.read())

In [ ]:
import json

In [ ]:
with open ('train.txt', "w") as f:
    f.write(json.dumps(ext_summary))

In [ ]:
test.to_csv('test_ext.csv', sep='\t')
test.head()

## EXTRACTIVE TO ABSTRACTIVE

GPT2 Decoder

In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
modelgpt2 = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

In [ ]:
abs_summary = []

for index, row in tqdm(test.iterrows()):
    input = row['ext_summary']
#     print(input)
    input_ids = tokenizer.encode(input, return_tensors='tf')
    summ = modelgpt2.generate(input_ids, max_length=200)
    summ_decoded = tokenizer.decode(summ[0], skip_special_tokens=True)
    abs_summary.append(summ_decoded)

test['abs_summary'] = abs_summary

BEAM

In [ ]:
abs_summary = []

for index, row in tqdm(test.iterrows()):
    input = row['ext_summary']
#     print(input)
    input_ids = tokenizer.encode(input, return_tensors='tf')
    beam_output = modelgpt2.generate(
        input_ids, 
        max_length=200, 
        num_beams=5, 
        early_stopping=True
    )

    summ_decoded =tokenizer.decode(beam_output[0], skip_special_tokens=True)
    abs_summary.append(summ_decoded)

test['beam_abs_summary'] = abs_summary

In [ ]:
test.head()

TOP P

In [ ]:
abs_summary = []

for index, row in tqdm(test.iterrows()):
    input = row['ext_summary']
#     print(input)
    input_ids = tokenizer.encode(input, return_tensors='tf')

    # set seed to reproduce results. Feel free to change the seed though to get different results
    tf.random.set_seed(0)
    # deactivate top_k sampling and sample only from 92% most likely words
    sample_output = modelgpt2.generate(
        input_ids, 
        do_sample=True, 
        max_length=200, 
        top_p=0.92, 
        top_k=0
    )
    summ_decoded = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    abs_summary.append(summ_decoded)
test['top_P_abs_summary'] = abs_summary

In [ ]:
test.to_csv('test_ext.csv', sep='\t')
test.head()

ROUGE

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
def rogue_sc(b, a):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = scorer.score(b,a)
    # print(scores)
    return scores

In [ ]:
import numpy as np

rouge1 = []
rouge2 = []
# word_capt = []
for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['top_P_abs_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['top_P_abs_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ top_P_abs_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))

for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['ext_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['ext_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ Extractive_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))


for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['beam_abs_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['beam_abs_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ Beams_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))


for index, row in test.iterrows():
    rouge1.append(rogue_sc(row['targets'], row['abs_summary'])['rouge1'][1])
    rouge2.append(rogue_sc(row['targets'], row['abs_summary'])['rouge2'][1])
    # word_capt.append(rogue_sc(row['targets'], row['Summary annotated'])['rouge1'][1])
print("------ Beams_summary-------")
print("Average Rouge1 Score: ", 100*np.average(np.array(rouge1)))
print("Average Rouge2 Score: ", 100*np.average(np.array(rouge2)))

In [ ]:
test.to_csv('test_ext.csv', sep='\t')

In [ ]:
!cat test_ext.csv